In [2]:
class AdaGrad:
    def __init__(self, ir=0.01):
        self.ir = ir
        self.h = None
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            
            for key, val in params.items():
                self.h[key]=np.zeros_like(val)
        
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.ir * grads[key] / (np.sqrt(self.h[key]) + 1e-7)
        

- 학습률이 감소하는 AdaGrad 
- 매개변수중 많이 움직인 원소는 기울기를 낮춘다. 다시말해 원소마다 다르게 적용한다.

- 모멘톰과 AdaGrad를 결합한 adam
- 현재는 SGD와 Adam이 많이 사용된다.

# 가중치 초기값 설정!

- 만약 가중치를 전부 0으로 설정하면 오차역전파법에 따라 모두 같은 값이 전달된다.
- 가중치가 고르게 되어버리는 상황을 우선적으로 막아야 한다.

- Xavier 초기값  표준편차가 1/루트(앞층의 노드 수) 로 생성하는 것
- 선형 활성화 함수에 대해서 각 층에 따라 기울기 감소가 일어나지 않고 효율적으로 일어난다.
- 시그모이드 함수와 tanh함수에 대해서 효율을 보여준다.

- 반면 ReLU함수를 이용할땐 He 초깃값이 효율을 보여준다.
- ReLU는 음의 영역이 0이라 더 넓게 분포시키기 위해 표준편차가 루트(2/n)인 정규분포를 사용한다.

- ReLU 함수 -> He 초깃값
- simoid, tanh 함수 -> Xavier 초깃값

- 활성화함수를 선택할 때, 시그모이드함수나 tanh함수같은 경우 큰 값을 0~1 , -1~ 1 로 간격을 줄인다.
- 따라서 발산한다고 느껴질 정도로 큰 값의 차이가 작아지는데 이런 문제는 ReLU함수에선 일어나지 않는다.
- 따라서 일반적으로 ReLU를 사용하고 마지막에 시그모이드나 tanh함수를 사용하는게 낫다.

- 시그모이드함수는 기울기 손실 문제도 일어난다.
- 반면 렐루함수는 기울기 손실문제가 일어나지 않고 값의 중요성을 가진다.

## 초기값 문제가 배치 정규화로 해결된다.

- 배치 정규화 알고리즘의 장점 3가지
- 학습을 빨리 진행
- 초깃값에 크게 의존하지 않는다.
- 오버피팅을 억제한다.

- 각 층의 활성화 함수를 적절하게 배치하기 위해 사용한다.
- 활성화함수 앞, 뒤에 배치정규화를 실시한다.

- 가중치 감소 ( weight decay)
- 오버피팅을 억제하기 위해 사용되는 가중치 감소
- 큰 가중치 매개변수의 값이 오버피팅 문제를 야기하기에 가중치의 제곱 법칙을 손실함수에 더해 오버피팅 문제를 해결한다.
- 가중치가 클수록 페널티를 주는 방식

- 드롭아웃
- 마찬가지로 오버피팅을 억제하는 방식
- 훈련때는 데이터를 흘릴 때마다 삭제할 뉴런을 무작위로 선택하고, 시험 때는 모든 뉴런에 신호를 전달하는 방식
- 훈련때 은닉층의 뉴런을 무작위로 골라 삭제한다.
- 가중치 감소가 효율을 발휘하지만, 신경망 모델이 복잡해지면 대응하기 어려워진다. 그때 드롭아웃을 사용한다.


In [3]:
class Dropout:
    def __init__(self, dropout_ratio=0.5):
        self.dropout_ratio = dropout_ratio
        self.mask= None
        
    def forward(self, x, train_flg=True):
        if train_flg:
            self.mask= np.random.rand(*x.shape)>self.dropout_ratio
        else:
            return x* (1.0 - self.dropout_ratio)
    
    def backward(self,dout):
        return dout * self.mask

### 하이퍼퍼라미터
- 각 층의 뉴런 수, 배치 크기, 매개변수 갱신 신의 학습률, 가중치 감소등을 뜻한다.
- 배치 정규화를 써야한다.
- 배치 정규화는 전체 데이터가 아니라 배치로 뽑은 데이터에 대한 정규화이기에 훨씬 효율적이다.